In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D,MaxPooling2D,concatenate,AveragePooling2D,Dropout,Dense,Softmax,BatchNormalization,ReLU,GlobalAveragePooling2D,Dense
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy
import tensorflow_datasets as tfds
import cv2
from tensorflow.keras.utils import plot_model
import numpy as np

In [2]:
data,info = tfds.load(
    'tf_flowers',
    split=['train'],
    shuffle_files=True,
    batch_size=-1,
    as_supervised=True,
    with_info=True)

In [3]:
images,labels = tfds.as_numpy(data[0])

In [5]:
def dataset_maker(img_data,labels):
    images = []
    for img in img_data:
        img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        img = cv2.resize(img,dsize=(224,224),interpolation=cv2.INTER_AREA)
        img = img.astype('float32')
        img /= 255
        images.append(img)
    images = np.array(images)
    labels = labels.astype('uint32')
    return images,labels

In [6]:
images,labels = dataset_maker(images,labels)

In [7]:
train_ds = tf.data.Dataset.from_tensor_slices((images[:2900],labels[:2900])).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((images[2900:],labels[2900:])).batch(32)

In [9]:
class DenseLayer(keras.layers.Layer):
    def __init__(self, growth_rate):
        super(DenseLayer, self).__init__()
        self.conv1 = Conv2D(filters=4*growth_rate, kernel_size=1, strides=1, padding='same', activation='relu')
        self.conv2 = Conv2D(filters=growth_rate, kernel_size=3, strides=1, padding='same', activation='relu')

    def call(self, inputs):
        x = self.conv1(inputs)
        x = self.conv2(x)
        return x

class TransitionLayer(keras.layers.Layer):
    def __init__(self, num_filters):
        super(TransitionLayer, self).__init__()
        self.conv = Conv2D(filters=num_filters, kernel_size=1, strides=1, padding='same', activation='relu')
        self.avgpool = AveragePooling2D(pool_size=2, strides=2, padding='same')

    def call(self, inputs):
        x = self.conv(inputs)
        x = self.avgpool(x)
        return x

class DenseNet121(keras.Model):
    def __init__(self, num_classes):
        super(DenseNet121, self).__init__()
        
        self.conv1 = Conv2D(filters=64, kernel_size=7, strides=2, padding='same', activation='relu')
        self.maxpool1 = MaxPooling2D(pool_size=3, strides=2, padding='same')
        self.block1 = DenseBlock(num_layers=6, growth_rate=32)
        self.transition1 = TransitionLayer(num_filters=128)
        self.block2 = DenseBlock(num_layers=12, growth_rate=32)
        self.transition2 = TransitionLayer(num_filters=256)
        self.block3 = DenseBlock(num_layers=24, growth_rate=32)
        self.transition3 = TransitionLayer(num_filters=512)
        self.block4 = DenseBlock(num_layers=16, growth_rate=32)
        self.avgpool = GlobalAveragePooling2D()
        self.fc = Dense(num_classes, activation='softmax')

    def call(self, inputs):
        x = self.conv1(inputs)
        x = self.maxpool1(x)
        x = self.block1(x)
        x = self.transition1(x)
        x = self.block2(x)
        x = self.transition2(x)
        x = self.block3(x)
        x = self.transition3(x)
        x = self.block4(x)
        x = self.avgpool(x)
        outputs = self.fc(x)
        return outputs

class DenseBlock(tf.keras.layers.Layer):
    def __init__(self, num_layers, growth_rate):
        super(DenseBlock, self).__init__()
        self.layers_list = []
        for _ in range(num_layers):
            self.layers_list.append(DenseLayer(growth_rate))

    def call(self, inputs):
        x = inputs
        for layer in self.layers_list:
            y = layer(x)
            x = tf.concat([x, y], axis=-1)
        return x


In [10]:
model = DenseNet121(num_classes=5)

In [11]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optimizer = tf.keras.optimizers.Adam()

In [12]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [13]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images, training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    train_loss(loss)
    train_accuracy(labels, predictions)

In [14]:
@tf.function
def test_step(images, labels):
    predictions = model(images, training=False)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [15]:
EPOCHS = 5
with tf.device('/device:CPU:0'):
    for epoch in range(EPOCHS):
        train_loss.reset_states()
        train_accuracy.reset_states()
        test_loss.reset_states()
        test_accuracy.reset_states()

        for images, labels in train_ds:
            train_step(images, labels)
        for test_images, test_labels in test_ds:
            test_step(test_images, test_labels)
        print(
            f'Epoch {epoch + 1}, '
            f'Loss: {train_loss.result()}, '
            f'Accuracy: {train_accuracy.result() * 100}, '
            f'Test Loss: {test_loss.result()}, '
            f'Test Accuracy: {test_accuracy.result() * 100}')       

c:\Users\justf\miniconda3\envs\gpu\lib\site-packages\keras\backend.py:5582: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


Epoch 1, Loss: 1.6070692539215088, Accuracy: 25.34482765197754, Test Loss: 1.55287504196167, Test Accuracy: 26.623376846313477
Epoch 2, Loss: 1.505943775177002, Accuracy: 31.517240524291992, Test Loss: 1.4505360126495361, Test Accuracy: 29.870128631591797
Epoch 3, Loss: 1.4630377292633057, Accuracy: 34.413795471191406, Test Loss: 1.3896173238754272, Test Accuracy: 39.350650787353516
Epoch 4, Loss: 1.3532198667526245, Accuracy: 40.10344696044922, Test Loss: 1.2954232692718506, Test Accuracy: 41.68831253051758
Epoch 5, Loss: 1.2425270080566406, Accuracy: 46.48276138305664, Test Loss: 1.216689109802246, Test Accuracy: 46.88311767578125
